## Идеи

1. Необходимо классифицировать изображения на 12 классов. Используем простую сверточную сеть с двумя полносвязными слоями. На выходе - слой размера 12 с softmax. На входе - тензор батча изобржаний
2. Изображения очень похожи друг на друга. В центре - побег растения, фон - почва (камни, замля и т.д.)
3. Изображения не стандартного формата (вроде). Необходимо привести к одному масштабу. 
4. Так как фото сделаны сверху - нет строгой ориентации побега (однако, он центрирован) и нет строгого отношения размер изображения/размер побега. Поэтому точно классной идеей будет применить аугментацию изображений поворотом и масштабированием. Возможно, также сдвигом. ImageDataGenerator
5. В качестве доп. фичей можно применить гистограмму цветов в области побега
6. Обучить несколько независимых сетей на частях данных (допустим, 4), делать стекинг на тестовой выборке из их предсказаний. Сам стекинг не обучать (для скорости)
7. Попробовать дообучать несколько слоев ResNet (хотя, эта сеть обучена для слишком большого количества изображений)
8. После обучения смотреть матрицу ошибок, выделять дополнительно то, что можно дообучить в случае больших ошибок сети
9. Подумать над размером и стратификацией батча (видел батчи по 16-20 фото, но у нас 12 классов. Так, есть большая вероятность не попадания нескольких классов в батч)

## imports

In [1]:
import pandas as pd
import numpy as np
import os
import imageio

from keras.utils import plot_model
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm
import os
import numpy as np
import cv2
from glob import glob
from sklearn.decomposition import PCA

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# global vars

In [2]:
BATCH_SIZE = 16
EPOCHS = 60
RANDOM_STATE = 5
CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

# Создание сети

In [3]:
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

In [4]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act

In [5]:
def get_model():
    inp_img = Input(shape=(107, 107, 3))

    # 106
    conv1 = conv_layer(inp_img, 32, zp_flag=False)
    conv2 = conv_layer(conv1, 32, zp_flag=False)
    mp1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv2)
    # 51
    conv3 = conv_layer(mp1, 64, zp_flag=False)
    conv4 = conv_layer(conv3, 64, zp_flag=False)
    mp2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv4)
    # 23
    conv7 = conv_layer(mp2, 256, zp_flag=False)
    conv8 = conv_layer(conv7, 256, zp_flag=False)
    conv9 = conv_layer(conv8, 256, zp_flag=False)
    mp3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv9)
    # 9
    conv10 = conv_layer(mp3, 256, zp_flag=False)
    conv11 = conv_layer(conv10, 256, zp_flag=False)
    conv12 = conv_layer(conv11, 256, zp_flag=False)
    mp4 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv12)
    # 1
    # dense layers
    flt = Flatten()(mp4)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = Model(inputs=inp_img, outputs=out)

    
#     mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    mypotim = SGD(lr=1 * 1e-3, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model

In [6]:
def get_callbacks(filepath, patience=5):
    lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=1e-5, patience=patience, verbose=1)
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [lr_reduce, msave]

In [7]:
# # Архитектура MNIST, не применяю тут
# def gen_model():
#     model = Sequential([
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(51, 51, 3)),
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Dropout(rate=0.25),
#         Flatten(),
#         Dense(1024, activation='relu'),
#         Dense(12, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

## Методы для обучения и предсказания

In [8]:
def train_model(img, target):
    callbacks = get_callbacks(filepath='model_weight_Adam_mask.hdf5', patience=5)
#     callbacks = get_callbacks(filepath='model_weight_SGD106.hdf5', patience=5)
    gmodel = get_model()
#     gmodel.load_weights(filepath='model_weight_Adam106.hdf5')
    x_train, x_valid, y_train, y_valid = train_test_split(
                                                        img,
                                                        target,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=RANDOM_STATE,
                                                        stratify = target
                                                        )
    gen = ImageDataGenerator(
            rotation_range=360.,
            width_shift_range=0.3,
            height_shift_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
    )
    
    gmodel.fit_generator(gen.flow(x_train, y_train,batch_size=BATCH_SIZE),
               steps_per_epoch=10*len(x_train)/BATCH_SIZE,
               epochs=EPOCHS,
               verbose=1,
               shuffle=True,
               validation_data=(x_valid, y_valid),
               callbacks=callbacks
               )
    gmodel.save_weights('my_weights.hdf5')
#     

In [9]:
def test_model(img, label):
    gmodel = get_model()
#     gmodel.load_weights(filepath='model_weight_Adam106.hdf5')
#     gmodel.load_weights(filepath='night_weights.hdf5')
#     gmodel.load_weights(filepath='model_weight_SGD106.hdf5')
    prob = gmodel.predict(img, verbose=1)
    pred = prob.argmax(axis=-1)
    sub = pd.DataFrame({"file": label,
                         "species": [INV_CLASS[p] for p in pred]})
    sub.to_csv("sub.csv", index=False, header=True)

# чтение и предобработка данных

In [10]:
def img_reshape(img):
    img = imresize(img, (107, 107, 3))
    return img

In [11]:
def img_label(path):
    return str(str(path.split('\\')[-1]))

In [12]:
def img_class(path):
    return str(path.split('\\')[-2])

In [13]:
def fill_dict(paths, some_dict):
    text = ''
    if 'train' in paths[0]:
        text = 'Start fill train_dict'
    elif 'test' in paths[0]:
        text = 'Start fill test_dict'

    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = cv2.imread(p)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)    
        mask = cv2.inRange(hsv, (24, 50, 0), (55, 255, 140))
        img_m = cv2.bitwise_and(img, img, mask=mask)
#         img_tot = np.concatenate([img, hsv, mask[..., np.newaxis]], axis = -1)
        img_tot = img_m
#         img = np.concatenate([bgr, hsv, mask[..., np.newaxis]], axis=-1)
        
        img = img_reshape(img_tot)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))
        if 'train' in paths[0]:
            some_dict['class'].append(img_class(p))
    return some_dict

In [14]:
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('../../Data'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]

                if ext not in file_ext:
                    file_ext.append(ext)

                if 'train' in root:
                    path = os.path.join(root, f)
                    train_path.append(path)
                elif 'test' in root:
                    path = os.path.join(root, f)
                    test_path.append(path)
    train_dict = {
        'image': [],
        'label': [],
        'class': []
    }
    test_dict = {
        'image': [],
        'label': []
    }

    train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

## Собственно, чтение данных в словарь

In [15]:
train_dict, test_dict = reader()

Root:
../../Data
Dirs:
['NLP-HW3', 'test', 'train']
Root:
../../Data\NLP-HW3
Dirs:
['names']
Root:
../../Data\train
Dirs:
['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


Start fill train_dict:   0%|                                | 0/4750 [00:00<?, ?it/s]C:\Anaconda\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Start fill test_dict: 100%|#######################| 794/794 [00:07<00:00, 103.83it/s]


## Составление необходимых ndarray, передаваемых в модель

In [16]:
X_train = np.array(train_dict['image'])
y_train = to_categorical(np.array([CLASS[l] for l in train_dict['class']]))

In [17]:
X_test = np.array(test_dict['image'])
label = test_dict['label']

In [18]:
del test_dict
del train_dict

In [19]:
len(X_train[0][0])

107

## Обучение модели

In [20]:
train_model(X_train, y_train)

C:\Anaconda\lib\site-packages\keras\callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 107, 107, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 105, 105, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 105, 105, 32)      128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 105, 105, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 103, 103, 32)      9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 103, 103, 32)      128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 103, 103, 32)      0         
__________

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Epoch 1/60


 204/2375 [=>............................] - ETA: 2:18:31 - loss: 2.7940 - acc: 0.06 - ETA: 1:11:11 - loss: 3.0117 - acc: 0.09 - ETA: 48:44 - loss: 2.8931 - acc: 0.0833 - ETA: 37:28 - loss: 2.8755 - acc: 0.09 - ETA: 30:43 - loss: 2.8052 - acc: 0.10 - ETA: 26:13 - loss: 2.8126 - acc: 0.08 - ETA: 23:01 - loss: 2.7524 - acc: 0.10 - ETA: 20:37 - loss: 2.7969 - acc: 0.09 - ETA: 18:44 - loss: 2.7795 - acc: 0.11 - ETA: 17:13 - loss: 2.7434 - acc: 0.11 - ETA: 16:00 - loss: 2.7039 - acc: 0.12 - ETA: 14:58 - loss: 2.7025 - acc: 0.12 - ETA: 14:06 - loss: 2.6715 - acc: 0.12 - ETA: 13:21 - loss: 2.6906 - acc: 0.12 - ETA: 12:43 - loss: 2.6944 - acc: 0.12 - ETA: 12:09 - loss: 2.7067 - acc: 0.12 - ETA: 11:39 - loss: 2.6760 - acc: 0.12 - ETA: 11:13 - loss: 2.6895 - acc: 0.12 - ETA: 10:49 - loss: 2.6781 - acc: 0.12 - ETA: 10:27 - loss: 2.6864 - acc: 0.11 - ETA: 10:08 - loss: 2.6684 - acc: 0.12 - ETA: 9:51 - loss: 2.6514 - acc: 0.1335 - ETA: 9:35 - loss: 2.6379 - acc: 0.141 - ETA: 9:20 - loss: 2.6222 - a

...

1841/2375 [======================>.......] - ETA: 1:11 - loss: 0.1746 - acc: 0.941 - ETA: 1:11 - loss: 0.1745 - acc: 0.941 - ETA: 1:11 - loss: 0.1745 - acc: 0.941 - ETA: 1:11 - loss: 0.1745 - acc: 0.941 - ETA: 1:10 - loss: 0.1745 - acc: 0.941 - ETA: 1:10 - loss: 0.1744 - acc: 0.941 - ETA: 1:10 - loss: 0.1744 - acc: 0.941 - ETA: 1:10 - loss: 0.1743 - acc: 0.941 - ETA: 1:10 - loss: 0.1743 - acc: 0.941 - ETA: 1:10 - loss: 0.1743 - acc: 0.941 - ETA: 1:10 - loss: 0.1743 - acc: 0.941 - ETA: 1:10 - loss: 0.1744 - acc: 0.941 - ETA: 1:10 - loss: 0.1744 - acc: 0.941 - ETA: 1:10 - loss: 0.1744 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1742 - acc: 0.941 - ETA: 1:09 - loss: 0.1741 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1743 - acc: 0.941 - ETA: 1:09 - loss: 0.1742 - acc: 0.9

2056/2375 [========================>.....] - ETA: 51s - loss: 0.1729 - acc: 0.94 - ETA: 51s - loss: 0.1729 - acc: 0.94 - ETA: 51s - loss: 0.1729 - acc: 0.94 - ETA: 50s - loss: 0.1729 - acc: 0.94 - ETA: 50s - loss: 0.1728 - acc: 0.94 - ETA: 50s - loss: 0.1728 - acc: 0.94 - ETA: 50s - loss: 0.1728 - acc: 0.94 - ETA: 50s - loss: 0.1727 - acc: 0.94 - ETA: 50s - loss: 0.1727 - acc: 0.94 - ETA: 50s - loss: 0.1727 - acc: 0.94 - ETA: 50s - loss: 0.1727 - acc: 0.94 - ETA: 50s - loss: 0.1726 - acc: 0.94 - ETA: 50s - loss: 0.1726 - acc: 0.94 - ETA: 49s - loss: 0.1726 - acc: 0.94 - ETA: 49s - loss: 0.1726 - acc: 0.94 - ETA: 49s - loss: 0.1726 - acc: 0.94 - ETA: 49s - loss: 0.1725 - acc: 0.94 - ETA: 49s - loss: 0.1725 - acc: 0.94 - ETA: 49s - loss: 0.1725 - acc: 0.94 - ETA: 49s - loss: 0.1724 - acc: 0.94 - ETA: 49s - loss: 0.1724 - acc: 0.94 - ETA: 49s - loss: 0.1724 - acc: 0.94 - ETA: 49s - loss: 0.1726 - acc: 0.94 - ETA: 49s - loss: 0.1727 - acc: 0.94 - ETA: 48s - loss: 0.1727 - acc: 0.94 - ETA: 

2271/2375 [===========================>..] - ETA: 30s - loss: 0.1728 - acc: 0.94 - ETA: 30s - loss: 0.1727 - acc: 0.94 - ETA: 30s - loss: 0.1727 - acc: 0.94 - ETA: 30s - loss: 0.1727 - acc: 0.94 - ETA: 30s - loss: 0.1726 - acc: 0.94 - ETA: 30s - loss: 0.1726 - acc: 0.94 - ETA: 30s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1727 - acc: 0.94 - ETA: 29s - loss: 0.1727 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1727 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1726 - acc: 0.94 - ETA: 29s - loss: 0.1725 - acc: 0.94 - ETA: 28s - loss: 0.1725 - acc: 0.94 - ETA: 28s - loss: 0.1724 - acc: 0.94 - ETA: 28s - loss: 0.1724 - acc: 0.94 - ETA: 28s - loss: 0.1724 - acc: 0.94 - ETA: 28s - loss: 0.1724 - acc: 0.94 - ETA: 28s - loss: 0.1724 - acc: 0.94 - ETA: 28s - loss: 0.1724 - acc: 0.94 - ETA: 

2375/2375 [==============================] - ETA: 9s - loss: 0.1730 - acc: 0.9429 - ETA: 9s - loss: 0.1730 - acc: 0.943 - ETA: 9s - loss: 0.1730 - acc: 0.943 - ETA: 9s - loss: 0.1731 - acc: 0.943 - ETA: 9s - loss: 0.1730 - acc: 0.943 - ETA: 9s - loss: 0.1731 - acc: 0.943 - ETA: 9s - loss: 0.1731 - acc: 0.942 - ETA: 9s - loss: 0.1731 - acc: 0.943 - ETA: 9s - loss: 0.1731 - acc: 0.943 - ETA: 9s - loss: 0.1731 - acc: 0.943 - ETA: 8s - loss: 0.1732 - acc: 0.942 - ETA: 8s - loss: 0.1732 - acc: 0.942 - ETA: 8s - loss: 0.1733 - acc: 0.942 - ETA: 8s - loss: 0.1733 - acc: 0.942 - ETA: 8s - loss: 0.1734 - acc: 0.942 - ETA: 8s - loss: 0.1736 - acc: 0.942 - ETA: 8s - loss: 0.1736 - acc: 0.942 - ETA: 8s - loss: 0.1736 - acc: 0.942 - ETA: 8s - loss: 0.1735 - acc: 0.942 - ETA: 8s - loss: 0.1735 - acc: 0.942 - ETA: 8s - loss: 0.1734 - acc: 0.942 - ETA: 7s - loss: 0.1734 - acc: 0.942 - ETA: 7s - loss: 0.1734 - acc: 0.942 - ETA: 7s - loss: 0.1733 - acc: 0.942 - ETA: 7s - loss: 0.1733 - acc: 0.942 - ETA:

KeyboardInterrupt: 

In [21]:
test_model(X_test, label)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 106, 106, 7)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 104, 104, 32)      2048      
_________________________________________________________________
batch_normalization_13 (Batc (None, 104, 104, 32)      128       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 104, 104, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 102, 102, 32)      9248      
_________________________________________________________________
batch_normalization_14 (Batc (None, 102, 102, 32)      128       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 102, 102, 32)      0         
__________

In [28]:
# 0.6801

1. Получить фичи из своей модели или из натренированной Xception в виде флаттен слоя
https://www.kaggle.com/gaborfodor/seedlings-pretrained-keras-models
2. Сделать фичи на основе характеристик изображения по маске
https://www.kaggle.com/gaborvecsei/plant-seedlings-fun-with-computer-vision
3. Объединить все в один длинный вектор. Возможно, сделать PCA.
4. На полученных фичах обучить нейросеть, XGBoost, RF. Сделать стекинг.
5. Отправить сабмит.